LinearRegression_Phase6_การทดลองที่2_Direct  
ref : https://towardsdatascience.com/6-methods-for-multi-step-forecasting-823cbde4127a/

#Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv")
df.head()

In [ ]:
df.columns

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

In [ ]:
df['is_weekend'] = (df.index.dayofweek == 6).astype(int)
df.head()

ตัวแปรที่ไม่ได้มีความหมายเชิงลำดับเช่น dayofweek, month  
เพราะตัวแปรเหล่านี้เป็นข้อมูลเชิงกลุ่ม (Categorical) ไม่ใช่เชิงลำดับ (Ordinal) หากเราใส่เป็นตัวเลข 1-12 เข้าไปตรงๆ โมเดลจะเข้าใจผิดว่า "เดือนธันวาคม (12)" มีค่ามากกว่า "เดือนมกราคม (1)" ถึง 12 เท่า ซึ่งไม่เป็นความจริง การทำ One-Hot Encoding จะสร้างคอลัมน์ใหม่เป็น is_Jan, is_Feb,... เพื่อแก้ปัญหานี้

In [ ]:
df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df['is_Mon'] = (df.index.dayofweek == 0).astype(int)
df['is_Tue'] = (df.index.dayofweek == 1).astype(int)
df['is_Wed'] = (df.index.dayofweek == 2).astype(int)
df['is_Thu'] = (df.index.dayofweek == 3).astype(int)
df['is_Fri'] = (df.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df['is_Jan'] = (df.index.month == 1).astype(int)
df['is_Feb'] = (df.index.month == 2).astype(int)
df['is_Mar'] = (df.index.month == 3).astype(int)
df['is_Apr'] = (df.index.month == 4).astype(int)
df['is_May'] = (df.index.month == 5).astype(int)
df['is_Jun'] = (df.index.month == 6).astype(int)
df['is_Jul'] = (df.index.month == 7).astype(int)
df['is_Aug'] = (df.index.month == 8).astype(int)
df['is_Sep'] = (df.index.month == 9).astype(int)
df['is_Oct'] = (df.index.month == 10).astype(int)
df['is_Nov'] = (df.index.month == 11).astype(int)
df['is_Dec'] = (df.index.month == 12).astype(int)
df.head(3)

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df = df.reindex(columns=['sales_sum', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                         , 'year', 'time_index'
                         #, 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'ismarketday', 'isschoolday', 'holiday'
                         ])

df.head()

In [ ]:
# prompt: drop row that have missing value

df.dropna(inplace=True)
df.head()

In [ ]:
df.tail()

In [ ]:
# สร้าง Target สำหรับ Direct Method
forecast_horizon = 7
target_cols = []
for h in range(1, forecast_horizon + 1):
    col_name = f'sales_sum_t+{h}'
    df[col_name] = df['sales_sum'].shift(-h)
    target_cols.append(col_name)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# กำหนดลิสต์ Feature สุดท้าย
features = [
    'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7',
    'year', 'time_index'
    #, 'ismarketday', 'isschoolday', 'holiday'
]

In [ ]:
# เราจะสร้าง DataFrame ใหม่สำหรับการเทรนโดยการ dropna()
# เพื่อให้ทุกแถวมีข้อมูล X และ y ครบถ้วน
df_for_training = df.dropna()

X_train = df_for_training[features]
y_train = df_for_training[target_cols]

In [ ]:
# 3.1) เตรียม Input (X_test)
# เราจะใช้ Feature จาก "แถวสุดท้ายที่มีข้อมูล X ครบถ้วน" ก่อนที่จะถูก dropna ไป
# ซึ่งก็คือแถวที่ 8 จากท้ายสุดของ DataFrame ดั้งเดิม (index -8)
# แถวนี้คือข้อมูลล่าสุดที่เราสามารถใช้ทำนาย 7 วันข้างหน้าได้
X_test = df.iloc[[-1 - forecast_horizon]][features]

# 3.2) เตรียมค่าจริงเพื่อเปรียบเทียบ (y_test_actual)
# ค่าจริงที่เราจะเปรียบเทียบด้วยคือยอดขาย 7 วันสุดท้ายจริงๆ ของ DataFrame ดั้งเดิม
y_test_actual = df['sales_sum'].tail(7).values

print(f"\nTest instance to predict from (date: {X_test.index[0].date()})")
print(f"Actual values to compare against (shape: {y_test_actual.shape})")

In [ ]:
X_test.tail()

#Train Model

In [ ]:
base_model = LinearRegression()
model_direct = MultiOutputRegressor(base_model)

# เทรนโมเดลด้วยข้อมูลที่สมบูรณ์เท่านั้น
model_direct.fit(X_train, y_train)
print("\nDirect multi-output model trained successfully.")

# ทำนาย 7 วันข้างหน้าจาก X_test ที่เราเตรียมไว้
predictions_direct = model_direct.predict(X_test).flatten()
print("Direct 7-step forecast completed.")
print("\nPredicted values:", predictions_direct)

#Evaluate

In [ ]:
# Evaluate the Direct method predictions
rmse_direct = np.sqrt(mean_squared_error(y_test_actual, predictions_direct))
mae_direct = mean_absolute_error(y_test_actual, predictions_direct)
print(f"\nDirect Method RMSE: {rmse_direct:.2f}")
print(f"Direct Method MAE: {mae_direct:.2f}")

In [ ]:
# prompt: พลอตกราฟการประเมินผล แกน X = date, Y = sales_sum, Actual sales = เส้นทึบสีน้ำเงินมีจุดกลม, Direct  forecast = เส้นทึบสีแดงมีกากะบาท

# Plotting
plt.figure(figsize=(12, 6))

# Plot Actual Sales
plt.plot(df.index[-forecast_horizon:], y_test_actual, marker='o', linestyle='-', color='blue', label='Actual Sales')

# Plot Direct Forecast
# เนื่องจาก predictions_direct เป็น numpy array ไม่มี index date เราต้องสร้าง index ให้ตรงกับช่วง forecast
forecast_dates = pd.date_range(start=df.index[-forecast_horizon], periods=forecast_horizon, freq='D')
plt.plot(forecast_dates, predictions_direct, marker='x', linestyle='-', color='red', label='Direct Forecast')

plt.title('Actual vs. Direct Forecast Sales Evaluation')
plt.xlabel('Date')
plt.ylabel('Sales Sum')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
